# _ipyrad_ testing tutorial

### Getting started
Import _ipyrad_ and remove previous test files if they are already present

In [1]:
## import modules
import ipyrad as ip      ## for RADseq assembly
print ip.__version__     ## print version

## clear data from test directory if it already exists
import shutil
import os
if os.path.exists("./test_rad/"):
    shutil.rmtree("./test_rad/")

0.0.62


### Assembly and Sample objects

Assembly and Sample objects are used by _ipyrad_ to access data stored on disk and to manipulate it. Each biological sample in a data set is represented in a Sample object, and a set of Samples is stored inside an Assembly object. The Assembly object has functions to assemble the data, and stores a log of all steps performed and the resulting statistics of those steps. Assembly objects can be copied or merged to allow branching events where different parameters can subsequently be applied to different Assemblies going forward. Examples of this are shown below.

To create an Assembly object call `ip.Assembly` and pass a name for the data set. An Assembly object does not initially contain Samples, they will be created either by linking fastq files to the Assembly object if data are already demultiplexed, or by running `step1()` to demultiplex raw data files, as shown below.

In [2]:
## create an Assembly object called data1. 
data1 = ip.Assembly("data1")

## The object will be saved to disk using its assigned name
print "Assembly object named", data1.name

Assembly object named data1


### Modifying assembly parameters
All of the parameter settings are linked to an Assembly object, which has a set of default parameters when it is created. These can be viewed using the `get_params()` function. To get more detailed information about all paramteres use `ip.get_params_info()` or to select a single parameter use `ip.get_params_info(3)`. Assembly objects have a function `set_params()` that can be used to modify parameters. 

In [3]:
## modify parameters for this Assembly object
data1.set_params(1, "./test_rad")
data1.set_params(2, "./data/sim_rad1_R1_.fastq.gz")
data1.set_params(3, "./data/sim_rad1_barcodes.txt")
data1.set_params(7, 3)
data1.set_params(10, 'rad')

## print the new parameters to screen
data1.get_params()

  1   working_directory             ./test_rad                                   
  2   raw_fastq_path                ./data/sim_rad1_R1_.fastq.gz                 
  3   barcodes_path                 ./data/sim_rad1_barcodes.txt                 
  4   sorted_fastq_path                                                          
  5   restriction_overhang          ('TGCAG', '')                                
  6   max_low_qual_bases            5                                            
  7   N_processors                  3                                            
  8   mindepth_statistical          6                                            
  9   mindepth_majrule              6                                            
  10  datatype                      rad                                          
  11  clust_threshold               0.85                                         
  12  minsamp                       4                                            
  13  max_shared

### Starting data
If the data are already demultiplexed then fastq files can be linked directly to the Data object, which in turn will create Sample objects for each fastq file (or pair of fastq files for paired data). The files may be gzip compressed. If the data are not demultiplexed then you will have to run the step1 function below to demultiplex the raw data.

In [4]:
## This would link fastq files from the 'sorted_fastq_path' if present
## Here it does nothing b/c there are no files in the sorted_fastq_path
data1.link_fastqs()

### Step 1: Demultiplex the raw data files
This uses the barcodes information to demultiplex reads in data files found in the 'raw_fastq_path'. It will create a Sample object for each sample that will be stored in the Assembly object. The state of each sample will be set to 1, meaning that the sample has completed step 1 of the _ipyrad_ assembly.

In [5]:
## run step 1 to demultiplex the data
data1.step1()

## print the results for each Sample in data1
print data1.stats

      state  reads_raw  reads_filtered  clusters_total  clusters_kept  \
1A_0      1       1986             NaN             NaN            NaN   
1B_0      1       1980             NaN             NaN            NaN   
1C_0      1       1981             NaN             NaN            NaN   
1D_0      1       1965             NaN             NaN            NaN   
2E_0      1       1986             NaN             NaN            NaN   
2F_0      1       2009             NaN             NaN            NaN   
2G_0      1       1923             NaN             NaN            NaN   
2H_0      1       1982             NaN             NaN            NaN   
3I_0      1       1987             NaN             NaN            NaN   
3J_0      1       1989             NaN             NaN            NaN   
3K_0      1       2015             NaN             NaN            NaN   
3L_0      1       1965             NaN             NaN            NaN   

      hetero_est  error_est  reads_consens  
1A_0 

### Step 2: Filter reads 
If for some reason we wanted to execute on just a subsample of our data, we could do this by selecting only certain samples to call the `step2` function on. Because `step2` is a function of `data`, it will always execute with the parameters that are linked to `data`. 

In [8]:
## example of ways to run step 2 to filter and trim reads
#data1.step2("1B_0")             ## run on a single sample
#data1.step2(["1B_0", "1C_0"])   ## run on one or more samples
data1.step2()                    ## run on all samples, skipping finished ones

## print the results
print data1.stats

skipping, 1B_0 already edited. Use force=True to overwrite
skipping, 2H_0 already edited. Use force=True to overwrite
skipping, 3J_0 already edited. Use force=True to overwrite
skipping, 2E_0 already edited. Use force=True to overwrite
skipping, 1C_0 already edited. Use force=True to overwrite
skipping, 1A_0 already edited. Use force=True to overwrite
skipping, 2G_0 already edited. Use force=True to overwrite
      state  reads_raw  reads_filtered  clusters_total  clusters_kept  \
1A_0      2       1986            1986             NaN            NaN   
1B_0      2       1980            1980             NaN            NaN   
1C_0      2       1981            1981             NaN            NaN   
1D_0      2       1965            1965             NaN            NaN   
2E_0      2       1986            1986             NaN            NaN   
2F_0      2       2009            2009             NaN            NaN   
2G_0      2       1923            1923             NaN            NaN   
2H_

### Step 3: clustering within-samples

Let's imagine at this point that we are interested in clustering our data at two different clustering thresholds. We will try 0.90 and 0.85. First we need to make a copy the Assembly object. This will inherit the locations of the data linked in the first object, but diverge in any future applications to the object. Thus, they can share the same working directory, and will inherit shared files, but create divergently linked files within this directory. You can view the directories linked to an Assembly object with the `.dirs` argument, shown below.

In [9]:
## create a copy of our Assemlbly object
data2 = data1.copy(newname="test2")

## set clustering threshold to 0.90
data2.set_params(11, 0.90)

## look at inherited parameters
data2.get_params()

  1   working_directory             ./test_rad                                   
  2   raw_fastq_path                ./data/sim_rad1_R1_.fastq.gz                 
  3   barcodes_path                 ./data/sim_rad1_barcodes.txt                 
  4   sorted_fastq_path                                                          
  5   restriction_overhang          ('TGCAG', '')                                
  6   max_low_qual_bases            5                                            
  7   N_processors                  3                                            
  8   mindepth_statistical          6                                            
  9   mindepth_majrule              6                                            
  10  datatype                      rad                                          
  11  clust_threshold               0.9                                          
  12  minsamp                       4                                            
  13  max_shared

In [10]:
## run step 3 to cluster reads within samples using vsearch
data1.step3()  # ["2H_0", "2G_0"])

## print the results
print data1.stats

clustering 12 samples on 3 processors
      state  reads_raw  reads_filtered  clusters_total  clusters_kept  \
1A_0      3       1986            1986             100            100   
1B_0      3       1980            1980             100            100   
1C_0      3       1981            1981             100            100   
1D_0      3       1965            1965             100            100   
2E_0      3       1986            1986             100            100   
2F_0      3       2009            2009             100            100   
2G_0      3       1923            1923             100            100   
2H_0      3       1982            1982             100            100   
3I_0      3       1987            1987             100            100   
3J_0      3       1989            1989             100            100   
3K_0      3       2015            2015             100            100   
3L_0      3       1965            1965             100            100   

      hetero

In [11]:
## run step 3 to cluster reads within samples using vsearch
data2.step3()  # ["2H_0", "2G_0"])

## print the results
print data2.stats

clustering 12 samples on 3 processors
      state  reads_raw  reads_filtered  clusters_total  clusters_kept  \
1A_0      3       1986            1986             100            100   
1B_0      3       1980            1980             100            100   
1C_0      3       1981            1981             100            100   
1D_0      3       1965            1965             100            100   
2E_0      3       1986            1986             100            100   
2F_0      3       2009            2009             100            100   
2G_0      3       1923            1923             100            100   
2H_0      3       1982            1982             100            100   
3I_0      3       1987            1987             100            100   
3J_0      3       1989            1989             100            100   
3K_0      3       2015            2015             100            100   
3L_0      3       1965            1965             100            100   

      hetero

### Branching Assembly objects
And you can see below that the two Assembly objects are now working with several shared directories (working, fastq, edits) but with different clust directories (clust_0.85 and clust_0.9). 

In [12]:
print "data1 directories:"
for (i,j) in data1.dirs.items():
    print "{}\t{}".format(i, j)
    
print "\ndata2 directories:"
for (i,j) in data2.dirs.items():
    print "{}\t{}".format(i, j)
    
## TODO: raw and fastq dirs... 

data1 directories:
fastqs	/home/deren/Dropbox/ipyrad/tests/test_rad/fastq
edits	/home/deren/Dropbox/ipyrad/tests/test_rad/edits
clusts	/home/deren/Dropbox/ipyrad/tests/test_rad/edits/clust_0.85
working	/home/deren/Dropbox/ipyrad/tests/test_rad

data2 directories:
fastqs	/home/deren/Dropbox/ipyrad/tests/test_rad/fastq
edits	/home/deren/Dropbox/ipyrad/tests/test_rad/edits
clusts	/home/deren/Dropbox/ipyrad/tests/test_rad/edits/clust_0.9
working	/home/deren/Dropbox/ipyrad/tests/test_rad


In [13]:
data1.statsfiles

{'s1': '/home/deren/Dropbox/ipyrad/tests/test_rad/fastq/s1_demultiplex_stats.txt',
 's2': '/home/deren/Dropbox/ipyrad/tests/test_rad/edits/s2_rawedit_stats.txt',
 's3': '/home/deren/Dropbox/ipyrad/tests/test_rad/edits/clust_0.85/s3_cluster_stats.txt'}

### Example of plotting with _ipyrad_
There are a a few simple plotting functions in _ipyrad_ useful for visualizing results. These are in the module `ipyrad.plotting`. Below is an interactive plot for visualizing the distributions of coverages across the 12 samples in the test data set.  

In [14]:
import ipyrad.plotting as iplot

## plot for one or more selected samples
iplot.depthplot(data1, ["1A_0", "1B_0"])

## plot for all samples in data1
#iplot.depthplot(data1)

## save plot as pdf and html
iplot.depthplot(data1, outprefix="testfig")

### Step 4: Joint estimation of heterozygosity and error rate


In [15]:
## run step 4
data1.step4()

## print the results
print data1.stats

      state  reads_raw  reads_filtered  clusters_total  clusters_kept  \
1A_0      4       1986            1986             100            100   
1B_0      4       1980            1980             100            100   
1C_0      4       1981            1981             100            100   
1D_0      4       1965            1965             100            100   
2E_0      4       1986            1986             100            100   
2F_0      4       2009            2009             100            100   
2G_0      4       1923            1923             100            100   
2H_0      4       1982            1982             100            100   
3I_0      4       1987            1987             100            100   
3J_0      4       1989            1989             100            100   
3K_0      4       2015            2015             100            100   
3L_0      4       1965            1965             100            100   

      hetero_est     error_est  reads_consens  
1A

### Step 5: Consensus base calls


In [ ]:
#import ipyrad as ip

## reload autosaved data. In case you quit and came back 
#data1 = ip.load_dataobj("test_rad/data1.assembly")

In [ ]:
## run step 5
#data1.step5()

## print the results
#print data1.stats

### Quick parameter explanations are always on-hand

In [16]:
ip.get_params_info(10)


        (10) clust_threshold -------------------------------------------------
        Clustering threshold. 
        Examples:
        ----------------------------------------------------------------------
        data.setparams(10) = .85          ## clustering similarity threshold
        data.setparams(10) = .90          ## clustering similarity threshold
        data.setparams(10) = .95          ## very high values not recommended 
        data.setparams("clust_threshold") = .83  ## verbose
        ----------------------------------------------------------------------
        


### Log history 
A common problem after struggling through an analysis is that you find you've completely forgotten what parameters you used at what point, and when you changed them. The log history time stamps all calls to `set_params()`, as well as calls to `step` methods. It also records copies/branching of data objects.  

In [17]:
for i in data1.log:
    print i

('data1', '10/09/15 01:54:29', 'data1 created')
('data1', '10/09/15 01:54:31', '[1] set to ./test_rad')
('data1', '10/09/15 01:54:31', '[2] set to ./data/sim_rad1_R1_.fastq.gz')
('data1', '10/09/15 01:54:31', '[3] set to ./data/sim_rad1_barcodes.txt')
('data1', '10/09/15 01:54:31', '[7] set to 3')
('data1', '10/09/15 01:54:31', '[10] set to rad')
('data1', '10/09/15 01:54:35', 's1_demultiplexing:')
('data1', '10/09/15 01:58:24', 's2 rawediting on 1B_0')
('data1', '10/09/15 01:58:25', 's2 rawediting on 2H_0')
('data1', '10/09/15 01:58:25', 's2 rawediting on 3J_0')
('data1', '10/09/15 01:58:25', 's2 rawediting on 2E_0')
('data1', '10/09/15 01:58:25', 's2 rawediting on 1C_0')
('data1', '10/09/15 01:58:25', 's2 rawediting on 1A_0')
('data1', '10/09/15 01:58:25', 's2 rawediting on 2G_0')
('data1', '10/09/15 01:58:37', 's2 rawediting on 3L_0')
('data1', '10/09/15 01:58:37', 's2 rawediting on 2F_0')
('data1', '10/09/15 01:58:37', 's2 rawediting on 3I_0')
('data1', '10/09/15 01:58:37', 's2 raw

### Saving Assembly objects
Assembly objects can be saved and loaded so that interactive analyses can be started, stopped, and returned to quite easily. The format of these saved files is a serialized 'dill' object used by Python. Individual Sample objects are saved within Assembly objects. These objects to not contain the actual sequence data, but only link to it, and so are not very large. The information contained includes parameters and the log of Assembly objects, and the statistics and state of Sample objects. Assembly objects are autosaved each time an assembly `step` function is called, but you can also create your own checkpoints with the `save` command. 

In [18]:
## save assembly object
#ip.save_assembly("data1.p")

## load assembly object
#data = ip.load_assembly("data1.p")
#print data.name